In [1]:
from tensorflow import data

from keras import Sequential, losses, metrics, optimizers
from keras.backend import clear_session
from keras.layers import LSTM, InputLayer
from keras.models import load_model

import pandas as pd
import numpy as np
import os

import config as c


In [2]:
def csv_to_tensor(target=c.input, step=c.step, take=None):
    sub_dataset = []
    for file in os.listdir(target)[:take]:
        df = pd.read_csv(os.path.join(target, file))
        df = data.Dataset.from_tensor_slices(df)
        df = df.window(step, shift=1, drop_remainder=True).flat_map(
            lambda x: x.batch(c.BATCH_SIZE))
        sub_dataset.append(df)

    tensor = sub_dataset[0]
    for dataset in sub_dataset[1:]:
        tensor = tensor.concatenate(dataset)
    return tensor, os.listdir(target)[:take]


In [3]:
take = 100
inputs, _ = csv_to_tensor(target=c.input, take=take)
outputs, _ = csv_to_tensor(target=c.output, take=take)

print('Data imported from:', take, 'videos')


Data imported from: 100 videos


In [4]:
dataset = data.Dataset.zip((inputs, outputs))

train_dataset = dataset.take(4000).shuffle(c.BUFFER_SIZE).batch(
    c.BATCH_SIZE).prefetch(data.experimental.AUTOTUNE)
val_dataset = dataset.skip(4000).batch(
    c.BATCH_SIZE).prefetch(data.experimental.AUTOTUNE)


In [5]:
clear_session()

model = Sequential(name='food_frame_classification')
model.add(InputLayer(input_shape=(None, c.features)))
model.add(LSTM(64, return_sequences=True, activation='relu'))
model.add(LSTM(4, return_sequences=True, activation='relu'))
model.add(LSTM(1, return_sequences=True, activation='sigmoid'))

model = load_model('saved_model')

model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer=optimizers.Adam(),
              metrics=metrics.Accuracy(),
              run_eagerly=True)

model.summary()


Model: "food_frame_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, None, 64)          147712    
                                                                 
 lstm_1 (LSTM)               (None, None, 4)           1104      
                                                                 
 lstm_2 (LSTM)               (None, None, 1)           24        
                                                                 
Total params: 148,840
Trainable params: 148,840
Non-trainable params: 0
_________________________________________________________________


In [6]:
history = model.fit(train_dataset, validation_data=val_dataset, epochs=100)


Epoch 1/100
63/63 [==============================] - 107s 2s/step - loss: 0.6855 - accuracy: 0.2345 - val_loss: 0.7593 - val_accuracy: 0.1302
Epoch 2/100
63/63 [==============================] - 107s 2s/step - loss: 0.6315 - accuracy: 0.2629 - val_loss: 0.7761 - val_accuracy: 0.1272
Epoch 3/100
63/63 [==============================] - 109s 2s/step - loss: 0.6170 - accuracy: 0.2868 - val_loss: 0.7633 - val_accuracy: 0.1826
Epoch 4/100
63/63 [==============================] - 108s 2s/step - loss: 0.6211 - accuracy: 0.1997 - val_loss: 0.7740 - val_accuracy: 0.0860
Epoch 5/100
63/63 [==============================] - 116s 2s/step - loss: 0.6206 - accuracy: 0.2102 - val_loss: 0.7781 - val_accuracy: 0.1050
Epoch 6/100
63/63 [==============================] - 155s 2s/step - loss: 0.6164 - accuracy: 0.2257 - val_loss: 0.7947 - val_accuracy: 0.0868
Epoch 7/100
63/63 [==============================] - 102s 2s/step - loss: 0.6132 - accuracy: 0.2043 - val_loss: 0.7772 - val_accuracy: 0.0886
Epoch 

In [7]:
model.save('saved_model')


INFO:tensorflow:Assets written to: saved_model\assets


In [21]:
def csv_to_predict(target=c.input, step=c.step, take=None):
    sub_dataset = []
    for file in os.listdir(target)[:take]:
        df = pd.read_csv(os.path.join(target, file))
        df = data.Dataset.from_tensor_slices(df)
        df = df.window(step, drop_remainder=True).flat_map(
            lambda x: x.batch(c.BATCH_SIZE))
        sub_dataset.append(df)

    tensor = sub_dataset[0]
    for dataset in sub_dataset[1:]:
        tensor = tensor.concatenate(dataset)
    return tensor, os.listdir(target)[:take]


In [22]:
predict, _ = csv_to_predict(target=c.input, take=1)
print('Prediction of video', _[0], ':')

predict = predict.batch(1)
result = model.predict(predict)
result = np.round(result)
for i in result:
    print(i.T)


Prediction of video -24c2YiaChY.csv :
39/39 [==============================] - 3s 64ms/step
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1.]]
[[0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1.]]
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1.]]
[[0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1.]]
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1.]]
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1.]]
[[0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1